# Training and Deploying a Custom Image Classifier with Amazon SageMaker
### This script was provided by [gabehollombe-aws](https://github.com/gabehollombe-aws/jupyter-notebooks/blob/master/DL%20Intro%20Demo%20-%20Sagemaker%20Image%20Classifer%20via%20Transfer%20Learning.ipynb)

## Configure where to fetch our training data

All of our images live inside an S3 bucket, organized into folders in a structure similar to this:

```
my_training_classes
├── person
│   ├── han.jpg
│   ├── leia.jpg
|   ├── luke.jpg
│   └── . . .
└── ship
│   ├── millenium_falcon.jpg
│   ├── tie-fighter.jpg    
│   ├── x-wing.jpg
│   ├── . . .
└── . . .
```

In [60]:
# An S3 Bucket Name
data_bucket_name='pixelscrapper-user-content'

# A prefix name inside the S3 bucket containing sub-folders of images (one per label class)
dataset_name = 'kit-image' 

## Setting up the environment
Here we set up the linkage and authentication to AWS services

- The role used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
- A `session` variable that holds some configuration state for interacting with SageMaker from Python and contains some methods for preparing input data
- A reference to the Amazon sagemaker image classification docker image 

More info about the SageMaker built-in Image Classification algorithm here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html

In [61]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import image_uris

role = get_execution_role()
sess = sagemaker.Session()

print(sess.boto_session.client('sts').get_caller_identity()['Arn'])
print(role)

training_image = image_uris.retrieve(region=sess.boto_region_name, framework='image-classification', version="latest")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


arn:aws:sts::466363126778:assumed-role/AWSGlueServiceSageMakerNotebookRole/SageMaker
arn:aws:iam::466363126778:role/AWSGlueServiceSageMakerNotebookRole


## Preparing data for our model
Before we can train our model, we need to:

- Create some files that will teach SageMaker about the images in each of our classes
- Upload these additional files to S3
- Configure our model to use these files for training and validating

### Find the im2rec.py script on this system
The SageMaker image classifier algorithm needs to know about which images belong to which classes. We provide this data using either LST or RecordIO files. We'll use a python script called `im2rec.py` to create these files.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html#IC-inputoutput

In [62]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=pixelscrapper-user-content
env: DATASET_NAME=kit-image
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


### Get our training images from S3
In order to create training and validation RecordIO files, we need to download our images to our local filesystem.

In [85]:
# Pull our images from S3
!aws s3 sync s3://pixelscrapper-user-content/kit-image /tmp/PSTEST

download: s3://pixelscrapper-user-content/kit-image/user-111918/node-263672/snow-beautiful-clusters-kit-cluster-frame-winter-orange-green-teal-purple.jpg to ../../../tmp/PSTEST/user-111918/node-263672/snow-beautiful-clusters-kit-cluster-frame-winter-orange-green-teal-purple.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-263670/snow-beautiful-frames-kit-frame-cardboard-winter-teal-purple-white.jpg to ../../../tmp/PSTEST/user-111918/node-263670/snow-beautiful-frames-kit-frame-cardboard-winter-teal-purple-white.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-264109/so-grateful-for-you-kit-autumn-thanksgiving-family-friends-orange-mint-green-brown.jpg to ../../../tmp/PSTEST/user-111918/node-264109/so-grateful-for-you-kit-autumn-thanksgiving-family-friends-orange-mint-green-brown.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-264432/pocket-page-layout-templates-kit-template-purple-coral-cream-gray.jpg to ../../../t

download: s3://pixelscrapper-user-content/kit-image/user-111918/node-266773/everyday-is-caturday-solid-kit-paper-solid-orange-peach-yellow-green-blue-lavender.jpg to ../../../tmp/PSTEST/user-111918/node-266773/everyday-is-caturday-solid-kit-paper-solid-orange-peach-yellow-green-blue-lavender.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-267235/forever-and-always-kit-kit-love-valentine-red-transparent-orange-blue-beige.jpg to ../../../tmp/PSTEST/user-111918/node-267235/forever-and-always-kit-kit-love-valentine-red-transparent-orange-blue-beige.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-267244/forever-and-always-pattern-papers-kit-pattern-valentine-heart-love-paper-red-orange-green-blue-beige.jpg to ../../../tmp/PSTEST/user-111918/node-267244/forever-and-always-pattern-papers-kit-pattern-valentine-heart-love-paper-red-orange-green-blue-beige.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-267243/forever-and

download: s3://pixelscrapper-user-content/kit-image/user-111918/node-270837/fido-pocket-clusters-kit-pocket-card-cluster-journal-element-gold-green-coral-cream.jpg to ../../../tmp/PSTEST/user-111918/node-270837/fido-pocket-clusters-kit-pocket-card-cluster-journal-element-gold-green-coral-cream.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-271151/forever-and-always-dex-cards-kit-dex-card-scripture-red-gold-blue-cream-beige.jpg to ../../../tmp/PSTEST/user-111918/node-271151/forever-and-always-dex-cards-kit-dex-card-scripture-red-gold-blue-cream-beige.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-271271/fido-valentine-cards-kit-valentine-dog-pet-puppy-printable-card-red-gold-coral-cream-brown.jpg to ../../../tmp/PSTEST/user-111918/node-271271/fido-valentine-cards-kit-valentine-dog-pet-puppy-printable-card-red-gold-coral-cream-brown.jpg
download: s3://pixelscrapper-user-content/kit-image/user-111918/node-271506/forever-and-always-script

download failed: s3://pixelscrapper-user-content/kit-image/user-12203/node-11154 to ../../../tmp/PSTEST/user-12203/node-11154 [Errno 21] Is a directory: '/tmp/PSTEST/user-12203/node-11154.d8B8f6E0' -> '/tmp/PSTEST/user-12203/node-11154'
download: s3://pixelscrapper-user-content/kit-image/user-12203/node-11154/color-green-dark.jpg to ../../../tmp/PSTEST/user-12203/node-11154/color-green-dark.jpg
download failed: s3://pixelscrapper-user-content/kit-image/user-12203/node-11158 to ../../../tmp/PSTEST/user-12203/node-11158 [Errno 21] Is a directory: '/tmp/PSTEST/user-12203/node-11158.FcF045cA' -> '/tmp/PSTEST/user-12203/node-11158'
download failed: s3://pixelscrapper-user-content/kit-image/user-12203/node-11159 to ../../../tmp/PSTEST/user-12203/node-11159 [Errno 21] Is a directory: '/tmp/PSTEST/user-12203/node-11159.14b2c4C6' -> '/tmp/PSTEST/user-12203/node-11159'
download failed: s3://pixelscrapper-user-content/kit-image/user-12203/node-15071 to ../../../tmp/PSTEST/user-12203/node-15071 [E

download: s3://pixelscrapper-user-content/kit-image/user-12831/node-21734/sheilareid-ntjt-elements-preview1000x1000.jpg to ../../../tmp/PSTEST/user-12831/node-21734/sheilareid-ntjt-elements-preview1000x1000.jpg
download: s3://pixelscrapper-user-content/kit-image/user-12463/node-86140/bohemian-breeze-september-2015-blog-train-kit-red-orange-teal-purple-gray.jpg to ../../../tmp/PSTEST/user-12463/node-86140/bohemian-breeze-september-2015-blog-train-kit-red-orange-teal-purple-gray.jpg
download: s3://pixelscrapper-user-content/kit-image/user-12463/node-93831/christmas-time-craft-papers-kit-paper-winter-red-green-white-brown.jpg to ../../../tmp/PSTEST/user-12463/node-93831/christmas-time-craft-papers-kit-paper-winter-red-green-white-brown.jpg
download failed: s3://pixelscrapper-user-content/kit-image/user-12831/node-21738 to ../../../tmp/PSTEST/user-12831/node-21738 [Errno 21] Is a directory: '/tmp/PSTEST/user-12831/node-21738.C89dC792' -> '/tmp/PSTEST/user-12831/node-21738'
download: s3://p

...

download: s3://pixelscrapper-user-content/kit-image/user-148/node-225697/boo-papers-kit-halloween-candy-trick-or-treat-sweet-spooky-spook-ghost-ghosts-children-kids-baby.jpg to ../../../tmp/PSTEST/user-148/node-225697/boo-papers-kit-halloween-candy-trick-or-treat-sweet-spooky-spook-ghost-ghosts-children-kids-baby.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-216592/winter-in-the-tropics-solid-papers-kit-summer-topics-tropical-relax-refresh-heatwave-ice-cream-paper.jpg to ../../../tmp/PSTEST/user-148/node-216592/winter-in-the-tropics-solid-papers-kit-summer-topics-tropical-relax-refresh-heatwave-ice-cream-paper.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-224345/jack-jill-alphas-2-kit-nursery-rhyme-rhymes-song-songs-sing-childhood-nostalgia-remember-memories.jpg to ../../../tmp/PSTEST/user-148/node-224345/jack-jill-alphas-2-kit-nursery-rhyme-rhymes-song-songs-sing-childhood-nostalgia-remember-memories.jpg
download: s3://pixelscrapper-user

download: s3://pixelscrapper-user-content/kit-image/user-148/node-243704/around-the-world-alphas-kit-globe-earth-planet-travel-adventure-vacation-sail-boat-sea-ocean-water.jpg to ../../../tmp/PSTEST/user-148/node-243704/around-the-world-alphas-kit-globe-earth-planet-travel-adventure-vacation-sail-boat-sea-ocean-water.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-247397/kumbaya-alphas-kit-alpha-alphabet-letters-numbers-punctuation-wood-veneer-rough-natural-nature.jpg to ../../../tmp/PSTEST/user-148/node-247397/kumbaya-alphas-kit-alpha-alphabet-letters-numbers-punctuation-wood-veneer-rough-natural-nature.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-236391/true-love-traveller-s-notebook-inserts-kit-valentine-s-day-anniversary-wedding-celebration-couple.jpg to ../../../tmp/PSTEST/user-148/node-236391/true-love-traveller-s-notebook-inserts-kit-valentine-s-day-anniversary-wedding-celebration-couple.jpg
download: s3://pixelscrapper-user-content

download: s3://pixelscrapper-user-content/kit-image/user-148/node-265270/season-of-gratitude-patterned-papers-2-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg to ../../../tmp/PSTEST/user-148/node-265270/season-of-gratitude-patterned-papers-2-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-265271/season-of-gratitude-patterned-papers-3-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg to ../../../tmp/PSTEST/user-148/node-265271/season-of-gratitude-patterned-papers-3-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-267402/sparkling-season-solid-glitter-papers-kit-christmas-holiday-sparkle-red-yellow-green-lime-dark-aqua.jpg to ../../../tmp/PSTEST/user-148/node-267402/sparkling-season-solid-glitter-papers-kit-christmas-holiday-sparkle-red-yellow-green-lime-dark-aqua.jpg
download: s3://pixelscrapper-

download: s3://pixelscrapper-user-content/kit-image/user-148/node-62501/lovestruck-papers-kit-love-valentine-s-day-heart-cupid-red-mint-green-purple-light-pink-white-brown.jpg to ../../../tmp/PSTEST/user-148/node-62501/lovestruck-papers-kit-love-valentine-s-day-heart-cupid-red-mint-green-purple-light-pink-white-brown.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-267153/season-of-gratitude-snippets-stamps-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg to ../../../tmp/PSTEST/user-148/node-267153/season-of-gratitude-snippets-stamps-kit-autumn-fall-thankful-blessed-grateful-birds-love-gather.jpg
download: s3://pixelscrapper-user-content/kit-image/user-148/node-63078/love-at-first-sight-tags-kit-hearts-bicycle-riding-cycling-documenting-orange-mint-green-aqua-blue.jpg to ../../../tmp/PSTEST/user-148/node-63078/love-at-first-sight-tags-kit-hearts-bicycle-riding-cycling-documenting-orange-mint-green-aqua-blue.jpg
download: s3://pixelscrapper-user-conte

^Cmpleted 1.4 GiB/~2.0 GiB (82.5 MiB/s) with ~1005 file(s) remaining (calculating...)


### Create RecordIO files from our training images
The `im2rec.py` script can create LST files and/or RecordIO files from our training data. 

More info here: https://mxnet.incubator.apache.org/versions/master/faq/recordio.html

In [14]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes


echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
mugs 0
notebooks 1
shoes 2
tennis balls 3
Creating RecordIO files
Creating .rec file from /tmp/testing_train.lst in /tmp
time: 0.06730914115905762  count: 0
Creating .rec file from /tmp/testing_test.lst in /tmp
time: 0.013591766357421875  count: 0
-rw-rw-r-- 1 ec2-user ec2-user 1.3M Feb 15 22:18 testing_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 3.5M Feb 15 22:18 testing_train.rec


rm: cannot remove ‘*.rec’: No such file or directory
rm: cannot remove ‘*.lst’: No such file or directory


### Upload our training and test data RecordIO files so we can train with them
Now that we have our training and test .rec files, we upload them to S3 so SageMaker can use them for training

In [17]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = sess.default_bucket()

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

delete: s3://sagemaker-eu-west-1-466363126778/testing/train/testing_train.rec
delete: s3://sagemaker-eu-west-1-466363126778/testing/validation/testing_test.rec
upload: ../../../tmp/testing_train.rec to s3://sagemaker-eu-west-1-466363126778/testing/train/testing_train.rec
upload: ../../../tmp/testing_test.rec to s3://sagemaker-eu-west-1-466363126778/testing/validation/testing_test.rec


### Configure the data for our model training to use
Finally, we tell SageMaker where to find these RecordIO files to use for training

In [18]:
train_data = sagemaker.inputs.TrainingInput(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.inputs.TrainingInput(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

## Training
Now it's time to train our model!

### Create an image classifier object with some base configuration
More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator

In [19]:
s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    instance_count=1, 
    instance_type='ml.p2.xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

### Set some training hyperparameters

Finally, before we train, we provide some additional configuration parameters for the training.

More info here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

In [20]:
num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])

# Learn more about the Sagemaker built-in Image Classifier hyperparameters here: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.001,
        mini_batch_size=5,
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters

{'use_pretrained_model': 1,
 'image_shape': '3,224,224',
 'num_classes': 4,
 'num_training_samples': 41,
 'learning_rate': 0.001,
 'mini_batch_size': 5}

### Start the training
Train our model!

This will take some time because it's provisioning a new container runtime to train our model, then the actual training happens, then the trained model gets uploaded to S3 and the container is shut down.

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.fit

In [21]:
%%time

import time
now = str(int(time.time()))
training_job_name = 'IC-' + dataset_name.replace('_', '-') + '-' + now

image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_classifier.output_path}/{job.name}/output/model.tar.gz")

2021-02-11 02:01:28 Starting - Starting the training job...
2021-02-11 02:01:52 Starting - Launching requested ML instancesProfilerReport-1613008888: InProgress
......
2021-02-11 02:02:52 Starting - Preparing the instances for training.........
2021-02-11 02:04:18 Downloading - Downloading input data...
2021-02-11 02:04:54 Training - Downloading the training image......
2021-02-11 02:05:55 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[02/11/2021 02:05:46 INFO 140014841898112] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[02

[02/11/2021 02:06:58 INFO 140014841898112] Epoch[13] Train-accuracy=1.000000
[02/11/2021 02:06:58 INFO 140014841898112] Epoch[13] Time cost=2.820
[02/11/2021 02:06:59 INFO 140014841898112] Epoch[13] Validation-accuracy=1.000000
[02/11/2021 02:07:02 INFO 140014841898112] Epoch[14] Train-accuracy=1.000000
[02/11/2021 02:07:02 INFO 140014841898112] Epoch[14] Time cost=2.876
[02/11/2021 02:07:03 INFO 140014841898112] Epoch[14] Validation-accuracy=1.000000
[02/11/2021 02:07:07 INFO 140014841898112] Epoch[15] Train-accuracy=1.000000
[02/11/2021 02:07:07 INFO 140014841898112] Epoch[15] Time cost=3.014
[02/11/2021 02:07:07 INFO 140014841898112] Epoch[15] Validation-accuracy=1.000000
[02/11/2021 02:07:10 INFO 140014841898112] Epoch[16] Train-accuracy=1.000000
[02/11/2021 02:07:10 INFO 140014841898112] Epoch[16] Time cost=2.622
[02/11/2021 02:07:11 INFO 140014841898112] Epoch[16] Validation-accuracy=1.000000
[02/11/2021 02:07:14 INFO 140014841898112] Epoch[17] Train-accuracy=1.000000
[02/11/2021

## Deploy the trained model
Once a model has been trained, we can use the same `image_classifier` object to create a deployed, fully-managed endpoint.}

More info here: https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator.deploy

In [44]:
%%time
# Deploying a model to an endpoint takes a few minutes to complete

deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium',
)

-------------------!

AttributeError: can't set attribute

### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [22]:
deployed_endpoint.delete_endpoint()

## Calling a deployed endpoint from Python code

If you want to try using a deployed endpoint from Python, here's a function that you can use. It takes in a path to the image you'd like to classify, and a list of all the classes used for training.

In [47]:
import json
import numpy as np
import os
# print(deployed_endpoint.content_type)
# deployed_endpoint.__set__ = 'image/png'

def classify_deployed(file_name, classes):
    payload = None
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)

    deployed_endpoint.content_type = 'image/png'
    result = json.loads(deployed_endpoint.predict(payload, initial_args={'ContentType': 'image/png'}))
    best_prob_index = np.argmax(result)
    return (result)

classify_deployed("mug.png", ["mugs", "notebooks", "shoes", "tennis balls"])

[0.8232777118682861,
 0.12408780306577682,
 0.017346825450658798,
 0.03528760001063347]

## (Optional) Perform Hyperparameter Tuning

Often, you might not know which values for hyperparameters like `learning_rate` and `mini_batch_size` will yield acceptible results. Traditionally, this meant manually running many training jobs with different hyperparameter values, looking at each trained model's performance, and then picking a winner. 

This type of manual tuning is _very_ time consuming, so you can automate this process using automatic model tuning with SageMaker. Here's some example code to illustrate how to start one of these jobs using the SageMaker Python SDK.

More info here about automatic model tuning: https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html

More info about model tuning for the Image Classification algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/IC-tuning.html

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'learning_rate': ContinuousParameter(0.0001, 0.1),
                         'mini_batch_size': IntegerParameter(2, 32),
                        }

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(image_classifier,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=50,
                            max_parallel_jobs=3)

tuner.fit(inputs=data_channels, logs=True, include_cls_metadata=False)

## Great resources to continue your Deep Learning journey

[3Blue1Brown’s YouTube series on Neural Networks ~ 60 Minutes](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi)

[Fast.ai’s Practical Deep Learning for Coders ~ 14 Hours](http://www.fast.ai/)
    
[Amazon's Machine Learning University ~ More than 45 hours of courses, videos, and labs](https://aws.amazon.com/training/learning-paths/machine-learning/)
    
[Neural Networks and Deep Learning, by Michael Neilsen ~ 6 Chapter Book](http://neuralnetworksanddeeplearning.com/)

[Amazon SageMaker - Fully-managed Platform](https://aws.amazon.com/sagemaker/)
    
[@gabehollombe's](https://twitter.com/gabehollombe) deep learning tools and demos
- [Jupyter Notebooks](https://github.com/gabehollombe-aws/jupyter-notebooks)
- [Webcam S3 Uploader Tool](https://github.com/gabehollombe-aws/webcam-s3-uploader)
- [SageMaker Inference Web Tool](https://github.com/gabehollombe-aws/webcam-sagemaker-inference)


In [2]:
%%bash
ls

lost+found
mug.png
Sagemaker Test.ipynb
Untitled.ipynb


In [79]:
# import boto3

# s3 = boto3.resource('s3')
# bucket = s3.Bucket('pixelscrapper-sagemaker')
# image_index = 0
# classes = dict()

# for bucket_obj in bucket.objects.all():
#     if ".JPG" in bucket_obj.key:
#         file = bucket_obj.key
#         first_dir = file.index('/')
#         second_dir = file[first_dir+1:].index('/')
#         folder_name = file[first_dir+1:first_dir + second_dir]
        
#         if classes.get(folder_name) == None:
#             classes[folder_name] = len(classes)
        
#         print(f"{image_index}\t{classes[folder_name]}\t{file[first_dir+1:]}")
#         image_index += 1


In [106]:
import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('pixelscrapper-user-content')
image_index = 0
max_width = 0

for bucket_obj in bucket.objects.filter(Prefix='kit-image/'):
    file = bucket_obj.key
    if ".jpg" in file or ".png" in file: # only select images from db
        last = file.rindex('/')
        class_list = file[last+1:].split('-') # split file name into list, delimited by '-'
        
        if len(class_list) == 0:
            continue
            
        print(f"{image_index}\t", end="")
        for cls in class_list:
            max_width = max(max_width, len(class_list)) # find largest width for an image
            cls.removesuffix(".jpg") # Remove file extensions
            cls.removesuffix(".png")
            print(f"{cls}\t", end="")
            
        first_slash = file.index('/') # Remove first directory from path
        print(f"{file[first_slash + 1:]}")
        image_index += 1
        
    if image_index == 500: # limit search to 500 images
        break


0	

AttributeError: 'str' object has no attribute 'removesuffix'

In [104]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
# rm *.rec
# rm *.lst

echo "Creating RecordIO files"
#                 image.lst image_root_dir output.bin resize=256 label_width=4
python $IM2REC mylst.lst PSTEST lable_width=15 
# python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating RecordIO files


usage: im2rec.py [-h] [--list] [--exts EXTS [EXTS ...]] [--chunks CHUNKS]
                 [--train-ratio TRAIN_RATIO] [--test-ratio TEST_RATIO]
                 [--recursive] [--no-shuffle] [--pass-through]
                 [--resize RESIZE] [--center-crop] [--quality QUALITY]
                 [--num-thread NUM_THREAD] [--color {-1,0,1}]
                 [--encoding {.jpg,.png}] [--pack-label]
                 prefix root
im2rec.py: error: unrecognized arguments: lable_width=15
ls: cannot access *.rec: No such file or directory


CalledProcessError: Command 'b'# Use the IM2REC script to convert our images into RecordIO files\n\n# Clean up our working dir of existing LST and REC files\ncd $BASE_DIR\n# rm *.rec\n# rm *.lst\n\necho "Creating RecordIO files"\n#                 image.lst image_root_dir output.bin resize=256 label_width=4\npython $IM2REC mylst.lst PSTEST lable_width=15 \n# python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME\nls -lh *.rec\n'' returned non-zero exit status 2.

In [75]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
ls

brazil-npm-registry
core-js-banners
hsperfdata_role-agent
install_chronicled.py
install-rootless-docker.sh
jetty-localhost-9081-role-proxy-agent.war-_-any-8585371679333298435.dir
lib
lua_IEOMi7
mxnet_p36.err
npm-14556-2696305a
npm-15025-db52c56e
npm-15054-6be3b445
npm-15331-7bc30880
npm-15371-7b9da94e
npm-23920-3472eacb
npm-23949-8fd4063c
npm-23960-721d5136
npm-23971-60eb7220
npm-23982-dce5383b
PSTEST
rootless.tgz
testing
testing_classes
testing_test.idx
testing_train.idx
v8-compile-cache-500
wowlst_test.lst
wowlst_train.lst
yarn--1612191771061-0.37594055084732614
yarn--1612191771894-0.8927031976117452
yarn--1612191777695-0.6567305931684579
yarn--1612191978680-0.6035171750832184
yarn--1612191979391-0.533981448331605
yarn--1612192005134-0.3019131852117847
yarn--1612192005757-0.9559645765153615
yarn--1612192006379-0.6240362427652166
yarn--1612192053162-0.5197709190446702
yarn--1612192053880-0.13730936267570848
yarn--1612192059913-0.2462204254754774
yarn--1612192060531-0.5981403352611461
